# 2020 TO 2023 US Congress members' stock transactions analysis

In this project, I will perform exploratory data analysis on the data provided by [House Stock Watcher](https://housestockwatcher.com/api) on the US Congress members' stock transactions for the years 2020, 2021, 2022, and 2023. The dataset includes various columns providing information on disclosure dates, transaction dates, asset details, transaction types, amounts, and other relevant attributes.
I seek to uncover patterns, trends, and potential conflicts of interest within the data. As a result, I have developed thought-provoking questions that can guide my analysis and uncover meaningful insights:

1. What are the primary motivations driving Congress members' stock transactions?
   - How do demographic factors such as party affiliation, district representation, and state influence trading activities?
   - Are there noticeable patterns in trading behavior based on transaction types such as purchases or sales?
   - Can we identify any correlation such as transaction amount and party affiliations?  
2. Are there observable trends in the frequency and volumes of stock transactions over multiple years?
   - Do certain years exhibit higher trading activities among Congress members, and if so, what factors might contribute to these fluctuations?
   - How does transaction volume vary across different sectors and industries, are there sectors that Congress members show a particular interest in?
   - Can we identify any anomalies or spikes in trading activities warrant further investigation? 
3. Can we analyze the performance of Congress members' stock portfolios and identify any notable trends?
   - How do capital gain from stock transactions vary across different party affiliations?
   - Are there sectors that Congress members consistently realize higher capital gains, and what factors might explain these trends?
4. How can we cross-analyze trends between data dimensions to uncover deeper insights?
   - Are there correlations between transaction frequencies and demographic factors such as party affiliations or district representation?
   - How do sector-specific trends in stock transactions correlate with party affiliations?
   - Can we identify any systematic biases or patterns in trading activities that warrant further investigations?
     

In [1]:
import requests
import pandas as pd

In [2]:
api_url = "https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json"
response = requests.get(api_url)
data = response.json()
df = pd.DataFrame(data)
df.sample(3)

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,state,ptr_link,cap_gains_over_200_usd,industry,sector,party
7580,2020,06/12/2020,2020-05-11,self,--,"Los Angeles, CA Dept. of Water & Power",purchase,"$500,001 - $1,000,000",Scott H. Peters,CA52,CA,https://disclosures-clerk.house.gov/public_dis...,False,None,None,Democrat
8146,2022,06/19/2022,2022-06-01,None,SPGI,S&P Global Inc,sale_partial,"$1,001 - $15,000",Alan S. Lowenthal,CA47,CA,https://disclosures-clerk.house.gov/public_dis...,False,Finance: Consumer Services,Finance,Democrat
28,2022,07/28/2022,2022-06-21,None,AGNC,AGNC Investment Corp,purchase,"$1,001 - $15,000",Chris Jacobs,NY27,NY,https://disclosures-clerk.house.gov/public_dis...,False,Real Estate Investment Trusts,Real Estate,Republican
